In [ ]:
import torch
from torchvision import transforms
from PIL import Image
import os
import numpy as np
import cv2
from cycle_gan.models.pix2pix_model import Pix2PixModel
from cycle_gan.options.base_options import BaseOptions
from cycle_gan.data.produce_dataset import Pix2PixDataset
from torch.utils.data import Dataset, DataLoader
import argparse
from cycle_gan.util.visualizer import Visualizer
import time
from gans import Training, Options
from infer import Infer

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'data/preprocessing_outputs/transformed_images_labels/images'

# Trial run to see if pipeline works

In [ ]:
image_dir = "data/preprocessing_outputs/transformed_images_labels/images"
label_dir = "data/preprocessing_outputs/transformed_images_labels/labels"

# Create dataset and dataloader
dataset = Pix2PixDataset(image_dir, label_dir)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=4)

print(f"Loaded dataset with {len(dataset)} image-mask pairs.")

In [ ]:
input_dir = "data\preprocessing_outputs\\transformed_images_labels\images"   # Raw cell images
mask_dir = "data\preprocessing_outputs\\transformed_images_labels\labels"     # Corresponding segmentation masks
input_tuning_dir = "data\preprocessing_outputs\\tuning\\transformed_images_labels\images"
output_dir = "data\Training-unlabeled\Training-unlabeled\labels" # Output paired images
unlabeled_images_path = "data\preprocessing_outputs\\transformed_images_labels\images"  # Unlabeled images
pseudo_mask_output_path = "./dataset_pix2pix/test/generated_masks/"  # Where to save masks
pseudo_mask_output_tuning_path = "./data/dataset_pix2pix/tuning/generated_masks_2/"

# Load trained Pix2Pix model
options = Options()
training = Training()
infer = Infer()

opt = options.get_opt()
pix2pix_model = training.load_pix2pix_model(opt)
dataset = Pix2PixDataset(input_dir, mask_dir)
# Train pix2pix 
training.train_model(pix2pix_model, dataset, opt)

pix2pix_model = Pix2PixModel(opt)
checkpoint_path = "latest_generator.pth"

# Load state dictionary
#pix2pix_model.load_networks("latest")
pix2pix_model.netG.load_state_dict(torch.load(checkpoint_path))

pix2pix_model.netG.eval()
print("Model loaded successfully!")

# Generate pseudo-masks
infer.generate_pseudo_masks(pix2pix_model, input_tuning_dir, pseudo_mask_output_tuning_path)

print("Synthetic pseudo-masks added to training dataset!")
